# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [76]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [77]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,37.13,86,75,2.30,CA,1696191726
1,1,tralee,52.2704,-9.7026,52.70,92,100,5.84,IE,1696191726
2,2,yellowknife,62.4560,-114.3525,47.41,94,100,10.36,CA,1696191447
3,3,korem,12.5058,39.5228,58.35,64,6,3.85,ET,1696191726
4,4,west island,-12.1568,96.8225,76.98,73,75,17.27,CC,1696191726


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [78]:
%%capture --no-display

# Configure the map plot
humidity = city_data_df["Humidity"].astype(int)

map_plot = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM",
                                    frame_width = 800, frame_height = 600,
                                    size = "Humidity", color="City", alpha=.75)
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [84]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 30) & (city_data_df["Max Temp"] > 8)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display your sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
78,78,tiksi,71.6872,128.8694,16.66,92,61,5.73,RU,1696191735,No hotel found
119,119,ust-nera,64.5667,143.2000,28.22,62,24,7.00,RU,1696191740,No hotel found
139,139,bilibino,68.0546,166.4372,23.95,87,100,1.07,RU,1696191743,No hotel found
149,149,susuman,62.7833,148.1667,26.98,92,15,3.00,RU,1696191744,No hotel found
177,177,talnakh,69.4865,88.3972,9.52,93,22,6.26,RU,1696191748,No hotel found
210,210,longyearbyen,78.2186,15.6401,26.44,59,75,17.27,SJ,1696191654,No hotel found
217,217,udachny,66.4167,112.4000,28.45,100,100,17.45,RU,1696191752,No hotel found
320,320,anadyr,64.7500,177.4833,27.10,93,0,2.24,RU,1696191764,No hotel found
377,377,aasiaat,68.7098,-52.8699,23.32,78,2,9.01,GL,1696191770,No hotel found
533,533,severo-yeniseyskiy,60.3725,93.0408,26.58,87,100,8.01,RU,1696191680,No hotel found


### Step 3: Create a new DataFrame called `hotel_df`.

In [85]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
78,78,tiksi,71.6872,128.8694,16.66,92,61,5.73,RU,1696191735,
119,119,ust-nera,64.5667,143.2000,28.22,62,24,7.00,RU,1696191740,
139,139,bilibino,68.0546,166.4372,23.95,87,100,1.07,RU,1696191743,
149,149,susuman,62.7833,148.1667,26.98,92,15,3.00,RU,1696191744,
177,177,talnakh,69.4865,88.3972,9.52,93,22,6.26,RU,1696191748,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [96]:
# Set parameters to search for a hotel
radius = 1000000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tiksi - nearest hotel: No hotel found
ust-nera - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
susuman - nearest hotel: No hotel found
talnakh - nearest hotel: No hotel found
longyearbyen - nearest hotel: No hotel found
udachny - nearest hotel: No hotel found
anadyr - nearest hotel: No hotel found
aasiaat - nearest hotel: No hotel found
severo-yeniseyskiy - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
78,78,tiksi,71.6872,128.8694,16.66,92,61,5.73,RU,1696191735,No hotel found
119,119,ust-nera,64.5667,143.2000,28.22,62,24,7.00,RU,1696191740,No hotel found
139,139,bilibino,68.0546,166.4372,23.95,87,100,1.07,RU,1696191743,No hotel found
149,149,susuman,62.7833,148.1667,26.98,92,15,3.00,RU,1696191744,No hotel found
177,177,talnakh,69.4865,88.3972,9.52,93,22,6.26,RU,1696191748,No hotel found
210,210,longyearbyen,78.2186,15.6401,26.44,59,75,17.27,SJ,1696191654,No hotel found
217,217,udachny,66.4167,112.4000,28.45,100,100,17.45,RU,1696191752,No hotel found
320,320,anadyr,64.7500,177.4833,27.10,93,0,2.24,RU,1696191764,No hotel found
377,377,aasiaat,68.7098,-52.8699,23.32,78,2,9.01,GL,1696191770,No hotel found
533,533,severo-yeniseyskiy,60.3725,93.0408,26.58,87,100,8.01,RU,1696191680,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [82]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE